In [3]:
import kfp
import kfp.dsl as dsl
from kfp.v2.dsl import (
    component,
    Input,
    Output,
    Dataset,
    Metrics,
    pipeline,
    Artifact,
)

In [ ]:
def drop_cols(df, cols_to_drop):
    df = df.drop(cols_to_drop, axis=1)
    return df

In [4]:
@component(packages_to_install=['pandas'])
def drop_cols(df_artifact: Input[Artifact], cols_to_drop_string: str, output_artifact: Output[Artifact])->None:
    import pandas as pd 
    import json 
    cols_to_drop = json.loads(cols_to_drop_string)
    df = pd.read_csv(df_artifact.path)
    df = df.drop(cols_to_drop, axis=1)
    df.to_csv(output_artifact.path)

In [ ]:
@component(packages_to_install = ['google-cloud-storage']
)
def download_gcs_file(
project : str,
bucket : str,
filepath : str,
output_file : Output[Artifact]
) -> None:
'''Downloads a file from GCS to a KFP Artifact component'''
from google.cloud import storage

client = storage.Client(project = project)
bucket = client.get_bucket(bucket)
blob = bucket.blob(filepath)

blob.download_to_filename(output_file.path)

In [ ]:
@pipeline('titanic_pipeline')
def create_pipleine(cols_to_drop_string: str):
    download_gcs_file_op = download_gcs_file(project_string, bukcet_string, filepath_string)

    drop_cols_op = drop_cols(download_gcs_file_op.output, cols_to_drop_string)
    